In [1]:
import pandas as pd

# Load the returns data
rets = pd.read_csv('data/returns.csv', index_col=0)
future_returns = rets[-252:]
returns = rets[-252*2:-252]

# We use the penultime year as the returns matrix. The last year of the dataset will be used to check the fit goodness.

#print(returns.tail())
#print(future_returns.head())

In [2]:
import pymc as pm
import numpy as np

# Convert returns to numpy array
returns_data = returns.values

# Define the Bayesian model
with pm.Model() as model:
    # Lewandowski-Kurowicka-Joe Cholesky covariance matrix. This is a probability distribution over positive definite matrices.
    L, corr, std = pm.LKJCholeskyCov('L', n=returns_data.shape[1], eta=50, sd_dist=pm.HalfNormal.dist(5.0), compute_corr=True)
    
    # Reconstruct the covariance matrix from L
    sigma = pm.Deterministic('sigma', L.dot(L.T))
    
    # Observed returns data
    returns_obs = pm.MvNormal('returns_obs', mu=np.zeros(returns_data.shape[1]), chol=L, observed=returns_data)
    
    # Perform inference
    trace = pm.sample(2000, return_inferencedata=False, tune=1000)

# Extract the posterior mean of the covariance matrix
posterior_cov_matrix = np.mean(trace['sigma'], axis=0)

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [L]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 192 seconds.


In [3]:
from sklearn.covariance import LedoitWolf, OAS

# Ledoit-Wolf shrinkage estimator
lw = LedoitWolf()
lw_cov_matrix = lw.fit(returns_data).covariance_

oas = OAS()
oas_cov_matrix = oas.fit(returns_data).covariance_

# We use two known estimators, the ledoit wolf and the oracle approximating.

In [4]:
from scipy.stats import multivariate_normal

# Convert future returns to numpy array
future_returns_data = future_returns.values

# We will use the log likelihood to check the goodness of fit.
def log_likelihood(cov_matrix, data):
    mean_returns = np.zeros(cov_matrix.shape[0])
    mvn = multivariate_normal(mean=mean_returns, cov=cov_matrix)
    return mvn.logpdf(data).mean()

# Calculate log-likelihood for each covariance matrix
bayesian_log_likelihood = log_likelihood(posterior_cov_matrix, future_returns_data)
ledoitwolf_log_likelihood = log_likelihood(lw_cov_matrix, future_returns_data)
oas_log_likelihood = log_likelihood(oas_cov_matrix, future_returns_data)
sample_log_likelihood = log_likelihood(returns.cov(), future_returns_data)

print(f"Bayesian Log-Likelihood: {bayesian_log_likelihood}")
print(f"Ledoit-Wolf Log-Likelihood: {ledoitwolf_log_likelihood}")
print(f"Ledoit-Wolf Log-Likelihood: {oas_log_likelihood}")
print(f"Sample covariance Log-Likelihood: {sample_log_likelihood}")

Bayesian Log-Likelihood: 27.69286479363813
Ledoit-Wolf Log-Likelihood: 27.502390445199936
Ledoit-Wolf Log-Likelihood: 27.512044369390207
Sample covariance Log-Likelihood: 27.487805936692986


In [5]:
# Above we can see that the Bayesian estimator is better than traditional measures